# Setup
- packages installation  
- imports

In [1]:
%pip install numpy pandas matplotlib opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import os
import random

# Image Processing

In [4]:
DATASET_CARS_BR_PATH = "./dataset_RodoSol-ALPR/images/cars-br" # Path to images of cars with old license plate model
DATASET_CARS_ME_PATH = "./dataset_RodoSol-ALPR/images/cars-me" # Path to images of cars with MERCOSUL license plate model
DATASET_MOTORCYCLES_BR_PATH = "./dataset_RodoSol-ALPR/images/motorcycles-br" # Path to images of motorcycles with old license plate model
DATASET_MOTORCYCLES_ME_PATH = "./dataset_RodoSol-ALPR/images/motorcycles-me" # Path to images of motorcycles with MERCOSUL license plate model

VERBOSE = True

In [5]:
def verbose_decorator(func):
    def wrapper(*args, **kwargs):
        if VERBOSE:
            print()
            print(func)
        return func(*args, **kwargs)
    return wrapper

In [6]:
@verbose_decorator
def sample_images(data_path: str, exclude_path: str, sample_size=None) -> list[tuple]:
    """
    sample_images randomly chooses a number of images, equal to the parameter sample_size, from a directory specified by data_path 
    excluding images already at directory specified by exclude_path. If sample_size is None then pick all images from directory.
    It returns a list containing tuples, each tuple with a opencv image and its OS path.
    """
    
    if exclude_path == "" or exclude_path is None: # If not informed then ignore excluding.
        excluded_images = []
    else:
        excluded_images= [data_path + '/' + file_name for file_name in os.listdir(exclude_path) if file_name.endswith(".jpg")]

    image_paths = [data_path + '/' + file_name for file_name in os.listdir(data_path) if file_name.endswith(".jpg") and file_name not in excluded_images]
    if sample_size is not None:
        image_paths = random.sample(image_paths, min(sample_size, len(image_paths)))
    
    images = [(cv2.imread(image_path), image_path) for image_path in image_paths]
    if VERBOSE:
        print(f"Excluded files from sampling: {excluded_images}")
        print(f"Sampled files: {image_paths}")
        
    
    return images

In [22]:
@verbose_decorator
def show_comparison(original_images, processed_images):
    """
    show_comparison shows a comparison side by side between original_images and processed_images
    """
    images = [(orig, proc) for orig, proc in zip(original_images, processed_images)]
    if VERBOSE:
        print(f"images zipped structure: {[(orig[1], proc[1]) for orig, proc in images]}")
        for orig, proc in images:
            print(f"{orig[1].split('/')[-1]} shape: {orig[0].shape}")
            print(f"{proc[1]} shape: {proc[0].shape}")
    
    fig, axes = plt.subplots(len(images), 2, figsize=(20, 10))
    ax = axes.ravel()
    
    ax_i = -1
    for orig, proc in images:
        ax_i += 1
        ax[ax_i].imshow(orig[0])
        ax[ax_i].set_title(orig[1].split('/')[-1] + " Original")
        ax[ax_i].set_axis_off()
        
        ax_i += 1
        ax[ax_i].imshow(proc[0])
        ax[ax_i].set_title(proc[1].split('/')[-1] + " Processed")
        ax[ax_i].set_axis_off()


In [15]:
@verbose_decorator
def sample_execute_save_show(data_path: str, save_path: str, algorithm: callable, algorithm_args: dict, show=False, pre_sample=None, sample_size=None):
    """
    sample_execute_save_show randomly sample images from directory specified by data_path ignoring 
    images already at directory specified by save_path, execute a algorithm of image processing which must return a single image,
    save processed images at a directory specified by save_path and then shows a comparison between the original and processed image.
    If presample contains a valid images names list then execute algorithm with this images.
    If sample_size is None then execute algorithm on the entire directory.
    """
    os.makedirs(save_path, exist_ok=True) # Create directory and dont raise error if already exists
    is_pre_sample = isinstance(pre_sample, list) and all([sample in os.listdir(data_path) for sample in pre_sample]) # If is a list and directory specified by data_path contain ALL samples.
    
    if VERBOSE:
        print(f"is pre sampled: {is_pre_sample}")
    
    # Defining algorithm input scope
    if is_pre_sample: # Images pre sampled
        original_images = pre_sample
    elif sample_size is None:
        original_images = sample_images(data_path, save_path, sample_size=None)
    else: # Sample Images
        original_images = sample_images(data_path, save_path, sample_size)

    processed_images = []
    for original_image, path in original_images:
        image_save_path = save_path + '/' + path.split('/')[-1]

        processed_image = algorithm(original_image, **algorithm_args)
        processed_images.append( (processed_image, image_save_path) )

        cv2.imwrite(image_save_path, processed_image)
        if VERBOSE:
            print(f"{path.split('/')[-1]} saved at {image_save_path}")

    if show:
        show_comparison(original_images, processed_images)

    return processed_images

# Applying Grayscale

In [11]:
SAVE_CARS_BR_GRAYSCALE_PATH = "./output/images/grayscale/cars-br" # Save path to images of cars with old license plate model converted to grayscale
SAVE_CARS_ME_GRAYSCALE_PATH = "./output/images/grayscale/cars-me" # Save path to images of cars with MERCOSUL license plate model converted to grayscale
SAVE_MOTORCYCLES_BR_GRAYSCALE_PATH = "./output/images/grayscale/motorcycles-br" # Save path to images of motorcycles with old license plate model converted to grayscale
SAVE_MOTORCYCLES_ME_GRAYSCALE_PATH = "./output/images/grayscale/motorcycles-me" # Save path to images of motorcycles with MERCOSUL license plate model converted to grayscale


In [14]:
def convert_to_grayscale(image):
    """
    Convert an OpenCV image to grayscale.

    Args:
        image (numpy.ndarray): The input OpenCV image.

    Returns:
        numpy.ndarray: The grayscale image.
    """
    grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return grayscale_image

In [16]:
if not os.path.exists("./output/images/grayscale/"): # If grayscale was not executed yet
    _ = sample_execute_save_show(data_path=DATASET_CARS_BR_PATH, 
                                save_path=SAVE_CARS_BR_GRAYSCALE_PATH,
                                algorithm=convert_to_grayscale,
                                algorithm_args=dict())

    _ = sample_execute_save_show(data_path=DATASET_CARS_ME_PATH, 
                                save_path=SAVE_CARS_ME_GRAYSCALE_PATH,
                                algorithm=convert_to_grayscale,
                                algorithm_args=dict())

    _ = sample_execute_save_show(data_path=DATASET_MOTORCYCLES_BR_PATH, 
                                save_path=SAVE_MOTORCYCLES_BR_GRAYSCALE_PATH,
                                algorithm=convert_to_grayscale,
                                algorithm_args=dict())

    _ = sample_execute_save_show(data_path=DATASET_MOTORCYCLES_ME_PATH, 
                                save_path=SAVE_MOTORCYCLES_ME_GRAYSCALE_PATH,
                                algorithm=convert_to_grayscale,
                                algorithm_args=dict())


<function sample_execute_save_show at 0x000001CE23382F70>
is pre sampled: False

<function sample_images at 0x000001CE426D3940>
Excluded files from sampling: []
Sampled files: ['./dataset_RodoSol-ALPR/images/cars-br/img_000001.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000002.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000003.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000004.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000005.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000006.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000007.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000008.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000009.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000010.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000011.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000012.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000013.jpg', './dataset_RodoSol-ALPR/images/cars-br/img_000014.jpg', './dataset_RodoSol-ALPR/images/cars-br

## Downscaling

In [36]:
SAVE_CARS_BR_DOWNSCALING_PATH = "./output/images/downscaling/cars-br" # Save path to downscaled images of cars with old license plate model
SAVE_CARS_ME_DOWNSCALING_PATH = "./output/images/downscaling/cars-me" # Save path to downscaled images of cars with MERCOSUL license plate model
SAVE_MOTORCYCLES_BR_DOWNSCALING_PATH = "./output/images/downscaling/motorcycles-br" # Save path to downscaled images of motorcycles with old license plate model
SAVE_MOTORCYCLES_ME_DOWNSCALING_PATH = "./output/images/downscaling/motorcycles-me" # Save path to downscaled images of motorcycles with MERCOSUL license plate model

DOWNSCALING_RATIO = 0.5

In [37]:
@verbose_decorator
def resize(image, ratio, interpolation_method_name, flag):
    width = int(image.shape[1] * ratio)
    height = int(image.shape[0] * ratio)
    dim = (width, height)

    if VERBOSE:
        print(f"interpolation method: {interpolation_method_name}")
        print(f"downscaling ratio: {str(ratio)}")
        print(f"original image shape: {image.shape}")
        print(f"new shape: {dim}")
        
    return cv2.resize(image, dim, interpolation=flag)

In [ ]:
INTERPOLATION_FLAGS = [
    ('bilinear', cv2.INTER_LINEAR),
    ('bicubic', cv2.INTER_CUBIC),
    ('lanczos', cv2.INTER_LANCZOS4)
]


for interpolation_method_name, flag in INTERPOLATION_FLAGS:
    _ = sample_execute_save_show(data_path=SAVE_CARS_BR_GRAYSCALE_PATH, 
                                save_path=SAVE_CARS_BR_DOWNSCALING_PATH + '/' + interpolation_method_name,
                                algorithm=resize, 
                                algorithm_args={"ratio": DOWNSCALING_RATIO, "interpolation_method_name": interpolation_method_name, "flag": flag},
                                sample_size=50)
